# Atrapar porcentajes quiniela

In [1]:
import bs4 as bs
import urllib
import urllib.request
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
import time
import string
import pprint

import io
from urllib.request import urlopen, Request
import requests

import http.client
from http.client import IncompleteRead

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

print (webdriver.__version__)

4.15.2


In [24]:
URL = "https://www.eduardolosilla.es/quiniela/ayudas/porcentajes"

options = webdriver.ChromeOptions()
service = ChromeService(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)
#print(driver)

driver.get(URL)



NUM_TOTAL_JORNADAS = 20
MAIN_BOX_INCREMENT_BETWEEN_MATCHES = 5
NUM_MATCHES_IN_A_JORNADA = 14
PLENO_15_ADDITIONAL_ROWS = 2

dfTotal = pd.DataFrame()
columns = columns=['Season', 'Jornada', 'Line', 'LocalTeamName', 'AwayTeamName', 'Result',
                   'PorcentajeQuinielistaLocal', 'PorcentajeQuinielistaDraw', 'PorcentajeQuinielistaAway',
                   'PorcentajeLaeLocal',         'PorcentajeLaeDraw',         'PorcentajeLaeAway',
                   'PorcentajeRealLocal',        'PorcentajeRealDraw',        'PorcentajeRealAway'
                  ]

jornadaList = []
jornada = -1
# for every "jornada"
while jornada < NUM_TOTAL_JORNADAS:
    jornada += 1
    
    time.sleep(randint(2,3))  
    
    #####################################
    # Click on the "jornada" tab
    #####################################
    select = Select(driver.find_element("id",'u-hidden-scrollbar'))
    select.select_by_value(str(jornada))
    
    time.sleep(randint(2,4))  
    
    
    #####################################
    # Find main Box
    #####################################    
    mainBoxes = driver.find_elements("xpath","//div[@class='c-caja_base m-no-fixed-size']")
    
    results = driver.find_elements("xpath","//button[@class='u-pronostico m-signo']")
    #print(results[0].text)
    
    for mainBox in mainBoxes:
        #print(mainBox.text)
        mainBoxTable = mainBox.text.split('\n')
        #print(cajaTable)  # ['1', 'R.SOCIEDAD - GIRONA', '1', 'X', '2', '2', 'LAS PALMAS - MALLORCA', '1', 'X', '2', '3', 'ATH.CLUB - R.MADRID', '1', 'X', '2', . . .

        jornadaBox = driver.find_elements("xpath","//span[@class='c-selector-horizontal__content__text']")
        #print(jornadaBox[0].text)
        saux = (jornadaBox[0].text).split(' - ')
        jornada_ = [int(s) for s in saux[0].split() if s.isdigit()][0]
        season_ = saux[1]
        
        porcentajesTable = driver.find_elements("xpath","//div[@class='c-porcentajes__fila__container u-clearfix ng-star-inserted']")
        #print(porcentajesTable[16].text)

        ii_mainBoxTable = 0 # be careful not incrementing ++ (by one)
        while(ii_mainBoxTable < (MAIN_BOX_INCREMENT_BETWEEN_MATCHES * NUM_MATCHES_IN_A_JORNADA)):
            
            line_ = mainBoxTable[ii_mainBoxTable]
            #print(line_)
            saux = (mainBoxTable[ii_mainBoxTable+1]).split(' - ')
            localTeamName_ = saux[0]
            awayTeamName_ = saux[1]
            #print(localTeamName_)
            #print(awayTeamName_)

            saux = porcentajesTable[int(line_)-1].text.split('\n')
            porcentajeQuinielistaLocal_ = saux[0]
            porcentajeQuinielistaDraw_  = saux[1]
            porcentajeQuinielistaAway_  = saux[2]
            
            saux = porcentajesTable[int(line_)-1 + (NUM_MATCHES_IN_A_JORNADA+PLENO_15_ADDITIONAL_ROWS)*1].text.split('\n')
            porcentajeLaeLocal_ = saux[0]
            porcentajeLaeDraw_  = saux[1]
            porcentajeLaeAway_  = saux[2]
            
            saux = porcentajesTable[int(line_)-1 + (NUM_MATCHES_IN_A_JORNADA+PLENO_15_ADDITIONAL_ROWS)*2].text.split('\n')
            porcentajeRealLocal_ = saux[0]
            porcentajeRealDraw_  = saux[1]
            porcentajeRealAway_  = saux[2]

            row = [season_, jornada_, line_, localTeamName_, awayTeamName_, 
                   results[int(ii_mainBoxTable/MAIN_BOX_INCREMENT_BETWEEN_MATCHES)].text,
                   porcentajeQuinielistaLocal_, porcentajeQuinielistaDraw_, porcentajeQuinielistaAway_,
                   porcentajeLaeLocal_,         porcentajeLaeDraw_,         porcentajeLaeAway_,
                   porcentajeRealLocal_,        porcentajeRealDraw_,        porcentajeRealAway_
                  ]
            
            jornadaList.append(row)
  
            ii_mainBoxTable += MAIN_BOX_INCREMENT_BETWEEN_MATCHES

        #print(jornadaList)

df = pd.DataFrame(jornadaList, columns=columns)
df

,Season,Jornada,Line,LocalTeamName,AwayTeamName,Result,PorcentajeQuinielistaLocal,PorcentajeQuinielistaDraw,PorcentajeQuinielistaAway,PorcentajeLaeLocal,PorcentajeLaeDraw,PorcentajeLaeAway,PorcentajeRealLocal,PorcentajeRealDraw,PorcentajeRealAway
0,23/24,1,1,R.SOCIEDAD,GIRONA,X,71,20,9,73,18,9,55,26,19
1,23/24,1,2,LAS PALMAS,MALLORCA,X,40,33,27,34,38,28,38,32,30
2,23/24,1,3,ATH.CLUB,R.MADRID,2,21,24,55,18,22,60,31,28,41
3,23/24,1,4,CELTA,OSASUNA,2,56,26,18,52,30,18,45,30,25
4,23/24,1,5,VILLARREAL,BETIS,2,60,23,17,49,29,22,53,25,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,23/24,21,10,LEVANTE,RACING S.,2,62,24,14,65,22,13,44,28,28
290,23/24,21,11,PAÍSES BAJOS,REP.IRLANDA,1,95,4,1,87,8,5,75,16,9
291,23/24,21,12,LETONIA,CROACIA,2,1,5,94,7,9,84,5,13,82
292,23/24,21,13,PORTUGAL,ISLANDIA,1,96,3,1,91,6,3,84,11,5


In [25]:
df

,Season,Jornada,Line,LocalTeamName,AwayTeamName,Result,PorcentajeQuinielistaLocal,PorcentajeQuinielistaDraw,PorcentajeQuinielistaAway,PorcentajeLaeLocal,PorcentajeLaeDraw,PorcentajeLaeAway,PorcentajeRealLocal,PorcentajeRealDraw,PorcentajeRealAway
0,23/24,1,1,R.SOCIEDAD,GIRONA,X,71,20,9,73,18,9,55,26,19
1,23/24,1,2,LAS PALMAS,MALLORCA,X,40,33,27,34,38,28,38,32,30
2,23/24,1,3,ATH.CLUB,R.MADRID,2,21,24,55,18,22,60,31,28,41
3,23/24,1,4,CELTA,OSASUNA,2,56,26,18,52,30,18,45,30,25
4,23/24,1,5,VILLARREAL,BETIS,2,60,23,17,49,29,22,53,25,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,23/24,21,10,LEVANTE,RACING S.,2,62,24,14,65,22,13,44,28,28
290,23/24,21,11,PAÍSES BAJOS,REP.IRLANDA,1,95,4,1,87,8,5,75,16,9
291,23/24,21,12,LETONIA,CROACIA,2,1,5,94,7,9,84,5,13,82
292,23/24,21,13,PORTUGAL,ISLANDIA,1,96,3,1,91,6,3,84,11,5


-----